In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Dataset YOLO format

In [ ]:
import os
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from skimage import measure
from matplotlib.patches import Rectangle
from IPython.display import clear_output

from utils.preprocess import *

In [ ]:
X_train = np.load('/content/drive/MyDrive/Covid19/IMG_npy/Seg_opa/X_train_Seg_op.npy')
Y_train = np.load('/content/drive/MyDrive/Covid19/IMG_npy/Seg_opa/Y_train_Seg_op.npy')

X_test = np.load('/content/drive/MyDrive/Covid19/IMG_npy/Seg_opa/X_test_Seg_op.npy')
Y_test = np.load('/content/drive/MyDrive/Covid19/IMG_npy/Seg_opa/Y_test_Seg_op.npy') 

In [ ]:
X_train = min_max_preprocessing(X_train)
X_test  = min_max_preprocessing(X_test)

X_train, mean, std = samplewise_preprocessing(X_train)
X_test = featurewise_preprocessing(X_test, mean, std)

In [ ]:
for i in tqdm(range(X_train.shape[0])):
  plt.imshow(X_train[i], cmap="gray")
  plt.axis('off')
  plt.savefig(f'/content/drive/MyDrive/Covid19/BoundingBox_dataset/Version_1/images/train/{i+1}.png', bbox_inches='tight', pad_inches=0)
  plt.cla()

for i in tqdm(range(X_test.shape[0])):
  plt.imshow(X_test[i], cmap="gray")
  plt.axis('off')
  plt.savefig(f'/content/drive/MyDrive/Covid19/BoundingBox_dataset/Version_1/images/valid/{i+1}.png', bbox_inches='tight', pad_inches=0)
  plt.cla()

In [ ]:
# Convert segmentation masks into YOLO labels using the Skimage library

for i in tqdm(range(Y_train.shape[0])):
  label_mask = measure.label(Y_train[i])
  props = measure.regionprops(label_mask)
  f = open(f'/content/drive/MyDrive/Covid19/BoundingBox_dataset/Version_1/labels/train/{i+1}.txt', 'w')
  l = []
  for prop in props:
    width = (prop.bbox[3]-prop.bbox[1])/Y_train.shape[1]
    height = (prop.bbox[2]-prop.bbox[0])/Y_train.shape[1]
    center_x = prop.centroid[1]/Y_train.shape[1]
    center_y = prop.centroid[0]/Y_train.shape[1]
    l.append(f'0 {center_x:.4f} {center_y:.4f} {width:.4f} {height:.4f}\n')
  f.writelines(l)
  f.close

for i in tqdm(range(Y_test.shape[0])):
  label_mask = measure.label(Y_test[i])
  props = measure.regionprops(label_mask)
  f = open(f'/content/drive/MyDrive/Covid19/BoundingBox_dataset/Version_1/labels/valid/{i+1}.txt', 'w')
  l = []
  for prop in props:
    width = (prop.bbox[3]-prop.bbox[1])/Y_test.shape[1]
    height = (prop.bbox[2]-prop.bbox[0])/Y_test.shape[1]
    center_x = prop.centroid[1]/Y_test.shape[1]
    center_y = prop.centroid[0]/Y_test.shape[1]
    l.append(f'0 {center_x:.4f} {center_y:.4f} {width:.4f} {height:.4f}\n')
  f.writelines(l)
  f.close

In [ ]:
def box_union(l, idxs):

  """
  Function to join boxes from a list in YOLO format.

  Inputs
  l: List with the bounding boxes in YOLO format
  idxs: List of indexes of the boxes to join
  """
  
  l_ = []
  for idx in idxs:
    l_.append(l[idx])
  mini = np.min(np.array(l_), axis=0)
  maxi = np.max(np.array(l_), axis=0)
  l.append([mini[0], mini[1], maxi[2], maxi[3]])
  return l

In [ ]:
#i = int(os.listdir('/content/drive/MyDrive/Covid19/BoundingBox_dataset/Version_2/labels/train/')[-1][0])
i = 8
fig, ax = plt.subplots()
while True:
  print(i)
  f = open(f'/content/drive/MyDrive/Covid19/BoundingBox_dataset/Version_2/labels/train/{i+1}.txt', 'w')
  label_mask = measure.label(Y_train[i])
  fig, ax = plt.subplots()
  ax.imshow(Y_train[i], cmap=plt.cm.gray)
  props = measure.regionprops(label_mask)
  l = []
  for j, prop in enumerate(props):
    minr, minc, maxr, maxc = prop.bbox
    l.append([minr, minc, maxr, maxc])
    bx = (minc, maxc, maxc, minc, minc)
    by = (minr, minr, maxr, maxr, minr)
    ax.plot(bx, by, '-', linewidth=2.5, label=str(j))
  ax.legend()
  plt.show()

  v = input('Ingrese los números de las bounding boxes que desea unir: ')

  if v=='salir':
    break

  if v!='':
    if v == 'ok':
      data= []
      for prop in props:
        width = (prop.bbox[3]-prop.bbox[1])/Y_train.shape[1]
        height = (prop.bbox[2]-prop.bbox[0])/Y_train.shape[1]
        center_x = prop.centroid[1]/Y_train.shape[1]
        center_y = prop.centroid[0]/Y_train.shape[1]
        data.append(f'0 {center_x:.4f} {center_y:.4f} {width:.4f} {height:.4f}\n')
      f.writelines(data)
      f.close()
      i+=1
      continue
    idxs = v.split('-')
    print(idxs)
    if len(idxs) > 1:
      p = []
      for idx in idxs:
        ind = [int(a) for a in idx.split(' ')]
        l = box_union(l, ind)
        for ind1 in ind:
          p.append(ind1)
      for ind2 in sorted(p, reverse=True):
        l.pop(ind2)
    else:
      ind = [int(a) for a in idxs[0].split(' ')]
      l = box_union(l, ind)
    ax.clear()
    ax.imshow(Y_train[i], cmap=plt.cm.gray)
    data= []
    for j, prop in enumerate(l):
      minr, minc, maxr, maxc = prop
      bx = (minc, maxc, maxc, minc, minc)
      by = (minr, minr, maxr, maxr, minr)
      ax.plot(bx, by, '-', linewidth=2.5, label=str(j))
      width = (maxc-minc)/Y_train.shape[1]
      height = (maxr-minr)/Y_train.shape[1]
      center_x = (maxc+minc)/Y_train.shape[1]
      center_y = (maxr+minr)/Y_train.shape[1]
      data.append(f'0 {center_x:.4f} {center_y:.4f} {width:.4f} {height:.4f}\n')
    f.writelines(data)
    f.close()
    plt.show()
    input("Press Enter to continue...")
    i+=1
    

# YOLO model

## Set-up

In [ ]:
# %cd '/content/drive/MyDrive'
# !git clone https://github.com/ultralytics/yolov5  # clone repo
%cd '/content/Drive/MyDrive/COVID_19/yolov5'
!pip install -r requirements.txt  # install dependencies

import torch 
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
with open('/content/drive/MyDrive/Covid19/BoundingBox_dataset/Version_3/data.yaml', 'w') as file:
  file.writelines(['train: /content/drive/MyDrive/Covid19/BoundingBox_dataset/Version_3/images/train/ \n',
                   'val: /content/drive/MyDrive/Covid19/BoundingBox_dataset/Version_3/images/valid/ \n', 
                   'nc: 1 \n',
                   'names: ["GGO"] \n'])

In [ ]:
%cat '/content/drive/MyDrive/Covid19/BoundingBox_dataset/Version_3/data.yaml'

In [ ]:
import yaml
with open('/content/drive/MyDrive/Covid19/BoundingBox_dataset/data.yaml', 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
%cat '/content/drive/MyDrive/Covid19/yolov5/models/yolov5x.yaml'

## Custom config file

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /content/drive/MyDrive/Covid19/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
%%writetemplate /content/drive/MyDrive/Covid19/yolov5/models/custom_yolov5x.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 1.33  # model depth multiple
width_multiple: 1.25  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, C3, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

## Train

In [ ]:
!python train.py --img 224 --batch 128 --epochs 500 --data '/content/drive/MyDrive/Covid19/BoundingBox_dataset/Version_3/data.yaml' --weights yolov5x.pt
#--cfg '/content/drive/MyDrive/Covid19/yolov5/models/custom_yolov5x.yaml' --weights '/content/drive/MyDrive/Covid19/yolov5/runs/train/exp/weights/best.pt'

In [ ]:
from IPython.display import Image
print("GROUND TRUTH TRAINING DATA:")
Image(filename='/content/drive/MyDrive/Covid19/yolov5/runs/train/exp4/test_batch0_labels.jpg', width=900)

In [ ]:
Image(filename='/content/drive/MyDrive/Covid19/yolov5/runs/train/exp4/test_batch0_pred.jpg', width=900)

# Test

In [ ]:
!python test.py --img 224 --batch 128 --data '/content/drive/MyDrive/Covid19/BoundingBox_dataset/Version_3/data.yaml' --weights '/content/drive/MyDrive/Covid19/yolov5/runs/train/exp5/weights/best.pt'